In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from skinet.utils.athena_tools import *
warnings.filterwarnings("ignore")

In [2]:
# matplotlib 한글 나오게 하기

import matplotlib as mpl
import matplotlib.pylab as plt2 

plt2.rcParams["font.family"] = "NanumGothincCoding"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (14, 4) 
mpl.rcParams['axes.unicode_minus'] = False 

In [26]:
ym = '202010' # 전월 
fm = '202009' # 최초 청구월 기준
ft = 20200901 # 최종 통화일 기준

In [4]:
# tm list header update 
conn = athena_connection()
drop_table_in_athena(conn, 'default.tm_list_header')
temp = load_from_athena(conn, '''
SELECT DISTINCT listid
     , listname
     , listtypecd
     , extracteddate
     , extractedtime
     , extractorid
     , requestcnt
     , extractedcnt 
     , isallocation
     , teamcd
     , extractedgbn
FROM telepro.tm_list_cat
WHERE listtypecd = '35'
  AND extractorid = 'K1170127'
''')
load_to_athena(conn, temp, 'default.tm_list_header')
del temp

<< Delete s3://adt-sandbox/_create/tables/default.tm_list_header
...tmp_db name = tmp_20201111_18_08_08_efa64ffbdc2162c06f35a98a721430f0726c892a
Start creating table
drop table if exists default.tmp_20201111_18_08_08_efa64ffbdc2162c06f35a98a721430f0726c892a
<< table not exist

        create  table         default.tmp_20201111_18_08_08_efa64ffbdc2162c06f35a98a721430f0726c892a
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT DISTINCT listid
     , listname
     , listtypecd
     , extracteddate
     , extractedtime
     , extractorid
     , requestcnt
     , extractedcnt 
     , isallocation
     , teamcd
     , extractedgbn
FROM telepro.tm_list_cat
WHERE listtypecd = '35'
  AND extractorid = 'K1170127'

.
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 5)
Elapsed Time = 14.22
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/382ab468-5fe3-4dd2-b483-5f7daca941b6 /home/skinet

In [5]:
conn = athena_connection()
magam = load_from_athena(conn, '''

SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE) AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , a.kunnr as kunnr_hash
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
--     , (SELECT ETAX_EMAIL FROM SAP.ZSDT4021 WHERE CTNUM = A.CTNUM AND RQNUM = A.RQNUM AND AMT_TYPE = 'F3' AND MANDT = '100') AS 전자계산서EMAIL
--     , (SELECT GROS_CODE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS GROSCODE
     , a.svc_str
     , a.pay_type
     , a.depo_type
     , a.line_type
     , a.pck_str
--     , (SELECT BIR_DATE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS 생년월일
     , SUBSTR(c.stcd1, 1 ,6) AS B_YMD
     , SUBSTR(c.stcd1, 7 ,1) AS SEX
     , c.stcd2
     , c.j_1kftbus AS uptae
     , c.j_1kftind AS jongmok
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , a.gad_pstlz
     , a.gad_add02
     , a.post_add02
     , b.result1
     , b.result2
     , b.result3
     , b.result4
     , b.result5
     , b.result6
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.sk1_yn
     , b.sk2_yn
     , b.adt1_yn
     , b.aedat
     , b.aenam
     , e.ctrt_id
FROM sap.zsdt4013      AS a 
LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn, dm_yn, sk1_yn, sk2_yn, adt1_yn, aedat, aenam, result1, result2, result3, result4, result5, result6 FROM sap.zsdt3748 WHERE usest = 'X') AS b
      ON a.ctnum = b.ctnum
LEFT JOIN (SELECT kunnr, stcd2, stcd1, telf2, name1, j_1kftbus, j_1kftind FROM sap.kna1 WHERE mandt = '100') AS c
      ON a.kunnr = c.kunnr
LEFT JOIN (select distinct ctnum_hash, ctnum, kunnr_hash, kunnr from adt_work.hash_info) d
       ON a.ctnum = d.ctnum_hash
LEFT JOIN (select distinct ctrt_id, ctnum from telepro.tb_nsok_mapping) e
       ON a.ctnum = e.ctnum
  WHERE a.mandt = '100' 
    AND a.stmon = \'{}\'  --년월 변경
    AND a.bukrs = '1000'
    AND a.cstat IN ('1','2');

'''.format(ym)) 

...tmp_db name = tmp_20201111_18_08_45_2160cb28e6fd1b7ed70d5a1b9226ab848754ee8e
Start creating table
drop table if exists default.tmp_20201111_18_08_45_2160cb28e6fd1b7ed70d5a1b9226ab848754ee8e
<< table not exist

        create  table         default.tmp_20201111_18_08_45_2160cb28e6fd1b7ed70d5a1b9226ab848754ee8e
        with (format='Parquet', parquet_compression='GZIP')
        as
        

SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE) AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , a.kunnr as kunnr_hash
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
--     , (SELECT ETAX_EMAIL FROM SAP.ZSD

In [6]:
from skinet.utils.athena_tools import *
conn = athena_connection()
tm = load_from_athena(conn, '''
WITH t1 AS (
SELECT s25csq, MAX(talkenddate) AS talkenddate
  FROM telepro.tb_contact
 WHERE listid in (SELECT DISTINCT listid FROM default.tm_list_header)
GROUP BY s25csq
)

, t2 AS (
SELECT s25csq, talkenddate, MAX(contactendtime) AS contactendtime
  FROM telepro.tb_contact
 WHERE listid in (SELECT DISTINCT listid FROM default.tm_list_header)
GROUP BY s25csq, talkenddate
)

, t3 AS (
SELECT s25csq, talkenddate, contactendtime, dialresultcd
  FROM telepro.tb_contact
 WHERE listid in (SELECT DISTINCT listid FROM default.tm_list_header)  
)

, t4 AS (
SELECT s25csq, COUNT(DISTINCT listid) AS contact_cnt
  FROM telepro.tb_contact
 WHERE listid in (SELECT DISTINCT listid FROM default.tm_list_header)  
GROUP BY s25csq
)

SELECT t1.s25csq, t1.talkenddate, t2.contactendtime, t3.dialresultcd, t4.contact_cnt
  FROM t1
LEFT JOIN t2 
       ON t1.s25csq = t2.s25csq
      AND t1.talkenddate = t2.talkenddate
LEFT JOIN t3
       ON t1.s25csq = t3.s25csq
      AND t1.talkenddate = t3.talkenddate
      AND t2.contactendtime = t3.contactendtime
LEFT JOIN t4
       ON t1.s25csq = t4.s25csq
''') 

...tmp_db name = tmp_20201111_18_10_38_05d5a5527da9f96c139b0cfd137f39e9f3c48e0d
Start creating table
drop table if exists default.tmp_20201111_18_10_38_05d5a5527da9f96c139b0cfd137f39e9f3c48e0d
<< table not exist

        create  table         default.tmp_20201111_18_10_38_05d5a5527da9f96c139b0cfd137f39e9f3c48e0d
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT s25csq, MAX(talkenddate) AS talkenddate
  FROM telepro.tb_contact
 WHERE listid in (SELECT DISTINCT listid FROM default.tm_list_header)
GROUP BY s25csq
)

, t2 AS (
SELECT s25csq, talkenddate, MAX(contactendtime) AS contactendtime
  FROM telepro.tb_contact
 WHERE listid in (SELECT DISTINCT listid FROM default.tm_list_header)
GROUP BY s25csq, talkenddate
)

, t3 AS (
SELECT s25csq, talkenddate, contactendtime, dialresultcd
  FROM telepro.tb_contact
 WHERE listid in (SELECT DISTINCT listid FROM default.tm_list_header)  
)

, t4 AS (
SELECT s25csq, COUNT(DISTINCT listid) AS contact_

In [7]:
magam.shape

(766882, 61)

In [8]:
magam = pd.merge(magam, tm, how='left', left_on='ctnum_hash', right_on='s25csq')

In [9]:
drop_col = ['kcalss', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp', 'intro_asp', 'itype_sdesc', 'svc_str', 'pay_type', 'depo_type',
            'result1', 'result2', 'result3', 'result4', 'result5', 'result6']
magam.drop(drop_col, axis=1, inplace=True)

In [10]:
magam_copy = magam.copy()

In [11]:
# magam = magam_copy.copy()

In [12]:
cond0 = magam['first_bildate'].str.contains(fm)                                                    # 최초청구월

In [13]:
temp1 = magam[cond0]
temp1.shape

(17348, 50)

In [14]:
temp1['telf2'] = temp1['telf2'].str.replace('-|/|' '|'  '', '', regex=True)
temp1['telf2'] = np.where(temp1['telf2'].str[0] != '0', "0"+temp1['telf2'], temp1['telf2'])
temp1['telf2'] = temp1['telf2'].str.replace(' ', '')
temp1['telf2'] = temp1['telf2'].str.replace('(', '')
temp1['telf2'] = temp1['telf2'].str.replace(')', '')

In [15]:
temp1['talkenddate'] = temp1['talkenddate'].str.replace('/', '', regex=True)
temp1['talkenddate'].fillna('00000000', inplace=True)
temp1['talkenddate'] = temp1['talkenddate'].astype(int)

In [16]:
temp1['구분'] = '신규'

In [17]:
org = pd.read_csv('/home/skinet/hojin/tm_upsell/기존list.csv')
magam_copy['ctnum'] = magam_copy['ctnum'].astype('int')
org.shape

(24883, 1)

In [18]:
gizon = org.merge(magam_copy, how='inner', left_on='ctnum', right_on='ctnum')

In [19]:
gizon['구분'] = '기존'

In [20]:
gizon['telf2'] = gizon['telf2'].str.replace('-|/|' '|'  '', '', regex=True)
gizon['telf2'] = np.where(gizon['telf2'].str[0] != '0', "0"+gizon['telf2'], gizon['telf2'])
gizon['telf2'] = gizon['telf2'].str.replace(' ', '')
gizon['telf2'] = gizon['telf2'].str.replace('(', '')
gizon['telf2'] = gizon['telf2'].str.replace(')', '')
gizon['telf2'] = gizon['telf2'].str.replace('.', '')

In [21]:
gizon['talkenddate'] = gizon['talkenddate'].str.replace('/', '', regex=True)
gizon['talkenddate'].fillna('00000000', inplace=True)
gizon['talkenddate'] = gizon['talkenddate'].astype(int)

In [22]:
gizon.shape

(23648, 51)

In [23]:
full = pd.concat([temp1, gizon], axis=0)

In [24]:
full.drop(['ctnum_hash', 'kunnr_hash', 'pck_str', 'b_ymd', 'sex', 'stcd2', 's25csq'], axis=1, inplace=True)

In [25]:
full = full.rename(columns={
      'kunnr' : '고객번호',
      'ctnum' : '계약번호',
      'cstat' : '계약상태',
      'gad_name' : '상호',
      'gad_start' : '경비개시일',
      'conn_start' : '계약개시일',
      'conn_end' : '계약종료일',
      'monexp_amt' : '월정료',
      'conn_type' : '계약타입',
      'ctype_l' : '채널(대)',
      'ctype_m' : '채널(중)',
      'ctype_s' : '채널(소)',
      'itype_l' : '업종(대)',
      'itype_m' : '업종(중)',
      'itype_s' : '업종(소)',
      'name1' : '이름1',
      'gad_add01' : '경비물건주소',
      'post_add01' : '우편물수령지주소1',
      'first_bildate' : '최초청구일',
      'telf2' : '전화번호',
      'line_type' : '관제회선구분',
      'uptae' : '업태',
      'jongmok' : '종목',
      'gros_code' : '그로스코드',
      'gross_type' : '그로스타입',
      'kvgr4' : '세그먼트',
      'gad_pstlz' : '우편번호',
      'gad_add02' : '경비물건주소상세',
      'post_add02' : '우편물수령지주소2',
      'sms_yn' : 'SMS동의',
      'email_yn' : 'EMAIL동의',
      'tm_yn' : 'TM동의',
      'dm_yn' : 'DM동의',
      'sk1_yn' : 'SK1동의',
      'sk2_yn' : 'SK2동의',
      'adt1_yn' : '마케팅수신동의',
      'aedat' : '작업일자',
      'aenam' : '변경자',
      'ctrt_id' : 'n사계약번호',
      'rqnum' : '변경요청번호',
      'ins_type' : '보험구분',
      'ins_amt' : '보험금액',
      'ins_amt_type' : '배상액구분',
      'ins_payment' : '보험납부금액',
      'ins_stat' : '보험현상태구분',
      'ins_stat_nm' : '보험현상태구분이름',
      'ins_amt_type_nm' : '배상액구분이름',
      'talkenddate' : '최종접촉일',
      'contactendtime' : '최종접촉시간',
      'dialresultcd' : '최종접촉결과',
      'contact_cnt' : '접촉회차'
})

In [26]:
full = full[
    ['구분', '고객번호', '계약번호', '계약상태', '상호', '경비개시일', '계약개시일', '계약종료일', 
     '월정료', '계약타입', '채널(대)', '채널(중)', '채널(소)', '업종(대)', '업종(중)', '업종(소)', 
           '이름1', '경비물건주소', '우편물수령지주소1', '최초청구일', '전화번호',
           '관제회선구분', '업태', '종목', '그로스코드', '그로스타입', '세그먼트', '우편번호', '경비물건주소상세',
           '우편물수령지주소2', 'SMS동의', 'EMAIL동의', 'TM동의', 'DM동의', 'SK1동의', 'SK2동의', '마케팅수신동의',
           '작업일자', '변경자', 'n사계약번호', '최종접촉일', '최종접촉시간', '최종접촉결과',
           '접촉회차']
]

In [27]:
full.to_csv('/home/skinet/hojin/tm_upsell/20년8월기존_&_9월신규DB추출_201111.csv'.format(ym), encoding='utf-8-sig')

In [27]:
conn = athena_connection()
telf = load_from_athena(conn, '''
WITH t1 AS (
SELECT ctnum, MAX(erdat) AS erdat
  FROM sap.zsdt3748
 WHERE usest='X'
GROUP BY ctnum

)

, t2 AS (
SELECT a.ctnum, a.erdat, MAX(b.erzet) AS erzet
  FROM t1 a
LEFT JOIN (SELECT ctnum, erdat, erzet FROM sap.zsdt3748) b
       ON a.ctnum = b.ctnum
      AND a.erdat = b.erdat
GROUP BY a.ctnum, a.erdat
)

, t3 AS (
SELECT a.*
     , b.tm_yn
  FROM t2 a
LEFT JOIN (SELECT ctnum, erdat, erzet, tm_yn FROM sap.zsdt3748) b
      ON a.ctnum = b.ctnum
     AND a.erdat = b.erdat
     AND a.erzet = b.erzet
)

SELECT DISTINCT a.*
     , b.kunnr
     , c.telf2
  FROM t3 a
LEFT JOIN (SELECT DISTINCT ctnum, kunnr FROM sap.zsdt3748) b
      ON a.ctnum = b.ctnum
LEFT JOIN (SELECT DISTINCT kunnr, telf2 FROM sap.kna1) c
       ON b.kunnr = c.kunnr
LEFT JOIN (SELECT DISTINCT ctnum, cstat FROM sap.zsdt4013 WHERE stmon = \'{}\') d
       ON a.ctnum = d.ctnum
 WHERE a.tm_yn = 'N'
   AND d.cstat IN ('1', '2');
'''
.format(ym))

...tmp_db name = tmp_20201116_15_53_33_0d4dd826a48d971dae5f625cf158233eecd668c0
Start creating table
drop table if exists default.tmp_20201116_15_53_33_0d4dd826a48d971dae5f625cf158233eecd668c0
<< table not exist

        create  table         default.tmp_20201116_15_53_33_0d4dd826a48d971dae5f625cf158233eecd668c0
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT ctnum, MAX(erdat) AS erdat
  FROM sap.zsdt3748
 WHERE usest='X'
GROUP BY ctnum

)

, t2 AS (
SELECT a.ctnum, a.erdat, MAX(b.erzet) AS erzet
  FROM t1 a
LEFT JOIN (SELECT ctnum, erdat, erzet FROM sap.zsdt3748) b
       ON a.ctnum = b.ctnum
      AND a.erdat = b.erdat
GROUP BY a.ctnum, a.erdat
)

, t3 AS (
SELECT a.*
     , b.tm_yn
  FROM t2 a
LEFT JOIN (SELECT ctnum, erdat, erzet, tm_yn FROM sap.zsdt3748) b
      ON a.ctnum = b.ctnum
     AND a.erdat = b.erdat
     AND a.erzet = b.erzet
)

SELECT DISTINCT a.*
     , b.kunnr
     , c.telf2
  FROM t3 a
LEFT JOIN (SELECT DISTINCT ctn

In [28]:
telf['telf2'] = telf['telf2'].str.replace('-|/|' '|'  '', '', regex=True)
telf['telf2'] = np.where(telf['telf2'].str[0] != '0', "0"+telf['telf2'], telf['telf2'])
telf['telf2'] = telf['telf2'].str.replace(' ', '')
telf['telf2'] = telf['telf2'].str.replace('(', '')
telf['telf2'] = telf['telf2'].str.replace(')', '')

In [29]:
telf.head()

,ctnum,erdat,erzet,tm_yn,kunnr,telf2
0,59107a04cf672210fa53ab89c1577ecd94a02f91397514...,20180514,105319,N,f582ee37301a459d9e3f5cafe2e73fb70309b5a57159f8...,01043530005
1,51cd0896072ffc6ed4c1a20387a3e4747219dc189ba8d6...,20180430,200854,N,3385b278f4340be563dcc6bc4b41fc02781ee08870abd6...,01084778857
2,c03b64e223a8d28e139125a173032da9a544713eef2464...,20190315,085312,N,450eeff6e63178b4a5332b29a9a54636aba64d33147114...,01039277757
3,01ba51983a96009a2b3bc5162121410170d5763e5c5617...,20190401,130000,N,364d7698c52b5f2da49dbb048f33b2eeeca6438c7eb7e4...,01044123400
4,ba7a37555369dd85c7fb56d37cc47cfd9e4fb1af5ab8a5...,20190401,130000,N,9c2768197fd217fa01189c3b47d5d9566bbd140ccaf9a2...,01045004121


In [30]:
telf[telf['telf2'] == '01046795648']

,ctnum,erdat,erzet,tm_yn,kunnr,telf2


In [41]:
final = pd.DataFrame()
final['TM거부'] = list(telf['telf2'].value_counts().index)

In [43]:
final.to_csv('TM거부.CSV', encoding='utf-8-sig')

In [48]:
final

,TM거부
0,0
1,01025018431
2,01000000000
3,01011111111
4,01096993167
5,01034197872
6,01000000001
7,0분리보관
8,01012345678
9,01038384306
